# LIF Master Analysis Notebook
## Data Cleanup + Chart Generation (Single Source of Truth)

This notebook is the **single source of truth** for all LIF data analysis and visualization.
Run all cells to:
1. Clean and validate the dataset
2. Generate all 24 charts in Light Mode
3. Export rich JSON statistics

**Palette:** `#F8FAFC` (Bg), `#9AA6B2` (Primary), `#EF4444` (Danger)

**Data Sources:**
- Charoenwong & Bernardi (2022) - SSRN 3944435
- Bybit Security Report (Nov 2025)
- Anthropic Red Team (Dec 2025)
- Rekt.news, DeFiHackLabs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
import seaborn as sns
import json
import os
import numpy as np
from datetime import datetime

# --- CONFIGURATION ---
COLORS = {
    'background': '#F8FAFC',
    'light_blue': '#D9EAFD',
    'mid_blue': '#BCCCDC',
    'dark_blue': '#9AA6B2',
    'text': '#475569',
    'danger': '#EF4444',
    'warning': '#F59E0B',
    'success': '#10B981',
    'purple': '#8B5CF6',
    'pink': '#EC4899',
}

plt.rcParams.update({
    'figure.facecolor': COLORS['background'],
    'axes.facecolor': COLORS['background'],
    'axes.edgecolor': COLORS['mid_blue'],
    'axes.labelcolor': COLORS['text'],
    'xtick.color': COLORS['text'],
    'ytick.color': COLORS['text'],
    'text.color': COLORS['text'],
    'font.family': 'sans-serif',
    'axes.titleweight': 'bold',
    'axes.titlepad': 20,
    'figure.dpi': 150,
})

# Dynamic Path Resolution
def find_project_root():
    cwd = os.getcwd()
    # Walk up until we find legitimate-intervention-framework
    while os.path.basename(cwd) != 'legitimate-intervention-framework' and cwd != '/':
        cwd = os.path.dirname(cwd)
    if cwd == '/':
        # Fallback - assume we're in scripts/analysis
        return os.path.dirname(os.path.dirname(os.getcwd()))
    return cwd

BASE_DIR = find_project_root()
DATA_PATH = os.path.join(BASE_DIR, 'data/refined/lif_exploits_final.csv')
VIZ_PATH = os.path.join(BASE_DIR, 'visualizations')
JSON_PATH = os.path.join(BASE_DIR, 'data/refined/lif_stats.json')
CLEANED_CSV = os.path.join(BASE_DIR, 'data/refined/lif_exploits_cleaned.csv')

os.makedirs(VIZ_PATH, exist_ok=True)
print(f"Project Root: {BASE_DIR}")
print(f"Data Path: {DATA_PATH}")

Project Root: /Users/elemoghenekaro/Desktop/tasks/legitimate-intervention-framework
Data Path: /Users/elemoghenekaro/Desktop/tasks/legitimate-intervention-framework/data/refined/lif_exploits_final.csv


In [2]:
# Load raw data
df = pd.read_csv(DATA_PATH)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

print(f"Initial records: {len(df)}")
print(f"Initial total loss: ${df['loss_usd'].sum():,.0f}")

Initial records: 765
Initial total loss: $132,747,759,722


In [3]:
# ============================================================
# PHASE 1: DATA CLEANING
# ============================================================
print("\n" + "="*60)
print("DATA CLEANING")
print("="*60)

# --- FIX 1: Merge Bybit Duplicates ---
bybit_mask = (df['protocol'].str.contains('Bybit|ByBit', case=False, na=False)) & \
             (df['date'] >= '2025-02-19') & (df['date'] <= '2025-02-22')
bybit_rows = df[bybit_mask]
if len(bybit_rows) > 1:
    print(f"\n[FIX 1] Found {len(bybit_rows)} Bybit records (Feb 2025) - Merging...")
    max_idx = bybit_rows['loss_usd'].idxmax()
    drop_idx = [i for i in bybit_rows.index if i != max_idx]
    df = df.drop(drop_idx)
    df.loc[max_idx, 'loss_usd'] = 1500000000
    df.loc[max_idx, 'protocol'] = 'Bybit'
    df.loc[max_idx, 'vector_category'] = 'Blind Signing / Social Engineering (Lazarus)'
    print("  → Merged to single record: $1.5B")

# --- FIX 2: Deduplicate Terra/Luna ---
terra_mask = (df['loss_usd'] >= 39_000_000_000) & (df['year'] == 2022)
if terra_mask.sum() > 1:
    print(f"\n[FIX 2] Dropping {terra_mask.sum() - 1} duplicate Terra events")
    df = df.drop(df[terra_mask].index[1:])

# --- FIX 3: Reclassify Vectors ---
vector_mapping = {
    'Phishing': 'Phishing / Social Engineering',
    'Access Control': 'Access Control / Key Compromise',
    'Hot Wallet Access Control': 'Access Control / Key Compromise',
    'Private Key Compromise': 'Access Control / Key Compromise',
    'Private Key Breach': 'Access Control / Key Compromise',
    'Logic Flaw': 'Logic Bug / Code Error',
    'Logic Bug': 'Logic Bug / Code Error',
    'Reentrancy': 'Logic Bug / Code Error',
    'Reentrancy Attack': 'Logic Bug / Code Error',
    'Precision Loss': 'Logic Bug / Code Error',
    'Overflow': 'Logic Bug / Code Error',
    'Unsafe Math': 'Logic Bug / Code Error',
    'Oracle Issue': 'Oracle / Price Manipulation',
    'Price Manipulation': 'Oracle / Price Manipulation',
    'Faulty Oracle': 'Oracle / Price Manipulation',
    'Flash Loan Attack': 'Flash Loan / Economic Exploit',
    'Flashloan': 'Flash Loan / Economic Exploit',
    'Economic Collapse': 'Economic / Systemic Failure',
    'Economic Failure': 'Economic / Systemic Failure',
    'CeFi Insolvency': 'Economic / Systemic Failure',
    'Rugpull': 'Rugpull / Exit Scam',
    'Admin Coup': 'Governance / Voting Exploit',
}

def reclassify_vector(v):
    if pd.isna(v) or v == 'Other':
        return 'Uncategorized'
    for key, val in vector_mapping.items():
        if key.lower() in v.lower():
            return val
    return v

df['vector_category'] = df['vector_category'].apply(reclassify_vector)
print(f"\n[FIX 3] Vector reclassification complete")

# --- FIX 4: Reclassify Chains ---
protocol_chain_map = {
    'Terra / Luna': 'Terra', 'Luna / UST': 'Terra', 'Terra Classic': 'Terra',
    'Celsius': 'CeFi', 'Voyager': 'CeFi', 'Mt. Gox': 'CeFi', 'FTX': 'CeFi',
    'Mixin': 'CeFi', 'DMM Bitcoin': 'CeFi', 'Bybit': 'CeFi', 'WazirX': 'CeFi',
    'BtcTurk': 'CeFi', 'Indodax': 'CeFi', 'CoinEx': 'CeFi', 'Poloniex': 'CeFi',
}
chain_mapping = {'Other': 'Multi-chain / Unknown'}

def reclassify_chain(row):
    protocol = str(row['protocol']) if pd.notna(row['protocol']) else ''
    chain = str(row['chain']) if pd.notna(row['chain']) else 'Unknown'
    for prot, ch in protocol_chain_map.items():
        if prot.lower() in protocol.lower():
            return ch
    return chain_mapping.get(chain, chain)

df['chain'] = df.apply(reclassify_chain, axis=1)
print(f"[FIX 4] Chain reclassification complete")

# --- FIX 5: Add LIF Relevance Column ---
lif_relevant_vectors = ['Access Control', 'Logic Bug', 'Oracle', 'Flash Loan', 
                        'Reentrancy', 'Price Manipulation', 'Governance']

def is_lif_relevant(row):
    v = str(row['vector_category']).lower()
    return any(term.lower() in v for term in lif_relevant_vectors)

df['is_lif_relevant'] = df.apply(is_lif_relevant, axis=1)
print(f"[FIX 5] LIF relevance tagging: {df['is_lif_relevant'].sum()} relevant records")

# Standardize protocol names
df['protocol'] = df['protocol'].replace({
    'ByBit': 'Bybit', 'Terra Classic': 'Terra / Luna', 'Luna / UST': 'Terra / Luna'
})

# Save cleaned CSV
df.to_csv(CLEANED_CSV, index=False)
print(f"\n✓ Cleaned data saved: {CLEANED_CSV}")
print(f"  Final records: {len(df)}")
print(f"  Final total loss: ${df['loss_usd'].sum():,.0f}")


DATA CLEANING

[FIX 1] Found 2 Bybit records (Feb 2025) - Merging...
  → Merged to single record: $1.5B

[FIX 2] Dropping 1 duplicate Terra events

[FIX 3] Vector reclassification complete
[FIX 4] Chain reclassification complete
[FIX 5] LIF relevance tagging: 438 relevant records

✓ Cleaned data saved: /Users/elemoghenekaro/Desktop/tasks/legitimate-intervention-framework/data/refined/lif_exploits_cleaned.csv
  Final records: 763
  Final total loss: $91,317,759,722


In [4]:
# ============================================================
# EXTERNAL DATA SOURCES (with citations)
# ============================================================

# Charoenwong & Bernardi (2022), SSRN 3944435, Table 2
CHAROENWONG_DATA = {
    'hack_types': {'Security Breach': 20, 'Human Error': 5, 'Agency Problem': 5},
    'total_incidents': 30, 'date_range': '2011-2021',
    'source': 'Charoenwong & Bernardi (2022), SSRN 3944435'
}

# Bybit Security Report (Nov 2025)
BYBIT_DATA = {
    'freezing_capability': {'No Freezing': 131, 'Confirmed Freezing': 16, 'Potential Freezing': 19},
    'total_chains': 166,
    'freezing_methods': {
        'Hardcoded': ['BNB', 'VeChain', 'Chiliz', 'VIC', 'XDC'],
        'Config File': ['Sui', 'Aptos', 'Harmony', 'Linea', 'Waves', 'EOS', 'Oasis'],
        'Smart Contract': ['HECO'],
    },
    'source': 'Bybit Security Research, Nov 2025'
}

# Anthropic Red Team Report (Dec 2025)
ANTHROPIC_DATA = {
    'ai_exploit_revenue': {
        'dates': ['2025-01', '2025-03', '2025-05', '2025-07', '2025-09', '2025-11', '2025-12'],
        'revenue_millions': [0.005, 0.02, 0.08, 0.32, 1.28, 3.5, 4.6],
    },
    'doubling_rate_months': 1.3, 'cost_per_scan_usd': 1.22,
    'contracts_scanned': 2849, 'zero_days_found': 2,
    'source': 'Anthropic Red Team, Dec 2025'
}

# Recovery rates from case studies
RECOVERY_DATA = {
    'by_time': {
        'Minutes': {'rate': 95, 'examples': ['HECO freeze']},
        'Hours': {'rate': 60, 'examples': ['Sui freeze']},
        'Days': {'rate': 30, 'examples': ['BNB recovery']},
        'Weeks': {'rate': 5, 'examples': ['Most exploits']},
    },
    'source': 'LIF Case Studies Analysis'
}

print("✓ External data loaded with sources")

✓ External data loaded with sources


In [5]:
# ============================================================
# CHART GENERATION - Part 1: External Data Charts
# ============================================================
print("\n" + "="*60)
print("GENERATING CHARTS")
print("="*60)

chart_count = 0

# C01: Hack Type Distribution (Charoenwong)
fig, ax = plt.subplots(figsize=(8, 8))
data = CHAROENWONG_DATA['hack_types']
colors = [COLORS['danger'], COLORS['warning'], COLORS['dark_blue']]
ax.pie(data.values(), labels=data.keys(), autopct='%1.0f%%', colors=colors, startangle=90)
ax.set_title('Hack Type Distribution (2011-2021)\nSource: Charoenwong & Bernardi')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c01_hack_type_distribution.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C01: Hack Type Distribution")

# C03: Code is Law Breakdown (Bybit)
fig, ax = plt.subplots(figsize=(8, 8))
data = BYBIT_DATA['freezing_capability']
colors = [COLORS['mid_blue'], COLORS['danger'], COLORS['warning']]
ax.pie(data.values(), labels=data.keys(), autopct='%1.1f%%', colors=colors, startangle=90, explode=(0, 0.05, 0.02))
ax.set_title('Blockchain Freezing Capability\nSource: Bybit (166 chains)')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c03_code_is_law_breakdown.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C03: Code is Law Breakdown")

# C04: Freezing Methodology
methods = ['Hardcoded', 'Config File', 'Smart Contract']
transparency, speed, flexibility = [90, 30, 80], [20, 50, 95], [10, 40, 90]
x = np.arange(len(methods))
width = 0.25
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width, transparency, width, label='Transparency', color=COLORS['light_blue'])
ax.bar(x, speed, width, label='Speed', color=COLORS['dark_blue'])
ax.bar(x + width, flexibility, width, label='Flexibility', color=COLORS['danger'])
ax.set_ylabel('Score (0-100)')
ax.set_title('Freezing Methodology Comparison')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c04_freezing_methodology.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C04: Freezing Methodology")

# C05: Recovery vs Speed
times = list(RECOVERY_DATA['by_time'].keys())
rates = [v['rate'] for v in RECOVERY_DATA['by_time'].values()]
colors_bars = [COLORS['success'], COLORS['light_blue'], COLORS['warning'], COLORS['danger']]
fig, ax = plt.subplots(figsize=(8, 6))
bars = ax.bar(times, rates, color=colors_bars)
ax.bar_label(bars, fmt='%d%%', padding=3)
ax.set_ylabel('Recovery Rate (%)')
ax.set_xlabel('Intervention Speed')
ax.set_title('Recovery Rate vs Intervention Speed')
ax.set_ylim(0, 110)
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c05_recovery_vs_speed.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C05: Recovery vs Speed")

# C06: AI Acceleration
dates = ['Jan', 'Mar', 'May', 'Jul', 'Sep', 'Nov', 'Dec']
revenue = ANTHROPIC_DATA['ai_exploit_revenue']['revenue_millions']
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(dates, revenue, marker='o', color=COLORS['danger'], linewidth=3, markersize=8)
ax.fill_between(dates, revenue, color=COLORS['danger'], alpha=0.1)
ax.set_yscale('log')
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: f'${y:.2f}M' if y < 1 else f'${y:.1f}M'))
ax.set_title(f'AI Exploit Revenue Acceleration (2025)\nDoubling every {ANTHROPIC_DATA["doubling_rate_months"]} months')
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c06_ai_exploit_acceleration.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C06: AI Acceleration")

# C07: AI Token Cost
models = ['Opus 4.0\n(Jun)', 'Sonnet 4.0\n(Aug)', 'Opus 4.5\n(Oct)', 'Current\n(Dec)']
costs = [100, 77, 50, 29.8]
fig, ax = plt.subplots(figsize=(8, 6))
bars = ax.bar(models, costs, color=[COLORS['dark_blue'], COLORS['mid_blue'], COLORS['light_blue'], COLORS['success']])
ax.bar_label(bars, fmt='%.1f%%', padding=3)
ax.set_ylabel('Relative Token Cost (%)')
ax.set_title('AI Token Cost Reduction (70.2% in 6 months)')
ax.axhline(100, color=COLORS['danger'], linestyle='--', alpha=0.3)
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c07_ai_token_cost.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C07: AI Token Cost")

# C08: AI Cost Metric
fig, ax = plt.subplots(figsize=(6, 4))
ax.text(0.5, 0.6, f'${ANTHROPIC_DATA["cost_per_scan_usd"]:.2f}', fontsize=48, fontweight='bold', ha='center', va='center', color=COLORS['dark_blue'])
ax.text(0.5, 0.35, 'Average Cost per Contract Scan', fontsize=14, ha='center', va='center', color=COLORS['text'])
ax.text(0.5, 0.2, f'{ANTHROPIC_DATA["contracts_scanned"]:,} contracts | {ANTHROPIC_DATA["zero_days_found"]} zero-days found', fontsize=10, ha='center', va='center', color=COLORS['mid_blue'])
ax.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c08_ai_cost_metric.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C08: AI Cost Metric")

print(f"\n  Part 1 complete: {chart_count} charts")


GENERATING CHARTS


  [1] C01: Hack Type Distribution
  [2] C03: Code is Law Breakdown
  [3] C04: Freezing Methodology
  [4] C05: Recovery vs Speed


  [5] C06: AI Acceleration
  [6] C07: AI Token Cost
  [7] C08: AI Cost Metric

  Part 1 complete: 7 charts


In [6]:
# ============================================================
# CHART GENERATION - Part 2: Conceptual Diagrams
# ============================================================

# C09: Freeze Timeline
events = [('07:46', 'Balancer exploit', '$121M'), ('07:52', 'Hypernative flagged', ''),
          ('08:07', 'V6 pools paused', '$19.3M saved'), ('08:14', 'SEAL 911 contacted', ''),
          ('11:01', 'Recovery Mode', ''), ('Next Day', 'Moonwell exploit', '$1M')]
fig, ax = plt.subplots(figsize=(12, 4))
y_pos = 0.5
for i, (time, event, amount) in enumerate(events):
    x = i / (len(events) - 1)
    color = COLORS['danger'] if 'exploit' in event.lower() else COLORS['success']
    ax.scatter(x, y_pos, s=200, c=color, zorder=5)
    ax.annotate(f'{time}\n{event}\n{amount}', (x, y_pos + 0.1), ha='center', va='bottom', fontsize=9,
                bbox=dict(boxstyle='round,pad=0.3', facecolor=COLORS['light_blue'], alpha=0.5))
ax.axhline(y_pos, color=COLORS['mid_blue'], linewidth=2, zorder=1)
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('Freeze First Timeline: Nov 3-4, 2025 (Balancer & Moonwell)')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c09_freeze_timeline.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C09: Freeze Timeline")

# C10: 4 Architectures of Control
fig, ax = plt.subplots(figsize=(10, 8))
architectures = [('HARDCODED', 'Protocol Layer', 'Requires Hard Fork', COLORS['danger']),
                 ('CONFIG-BASED', 'Validator Config', 'Node Restart', COLORS['warning']),
                 ('SMART CONTRACT', 'Admin Contract', 'On-chain Tx', COLORS['success']),
                 ('REACTIVE', 'GitHub Commit', 'Chain Upgrade', COLORS['purple'])]
for i, (name, layer, action, color) in enumerate(architectures):
    y = 0.8 - i * 0.2
    ax.add_patch(mpatches.FancyBboxPatch((0.05, y - 0.08), 0.25, 0.14, boxstyle="round,pad=0.02", facecolor=color, alpha=0.3))
    ax.add_patch(mpatches.FancyBboxPatch((0.35, y - 0.08), 0.25, 0.14, boxstyle="round,pad=0.02", facecolor=color, alpha=0.5))
    ax.add_patch(mpatches.FancyBboxPatch((0.65, y - 0.08), 0.3, 0.14, boxstyle="round,pad=0.02", facecolor=color, alpha=0.7))
    ax.text(0.175, y, name, ha='center', va='center', fontsize=10, fontweight='bold')
    ax.text(0.475, y, layer, ha='center', va='center', fontsize=10)
    ax.text(0.8, y, action, ha='center', va='center', fontsize=10)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('4 Architectures of Control')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c10_architectures_control.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C10: Architectures of Control")

# C13: 5 Regulatory Models (Table)
models_data = {
    'Model': ['Centralized', 'Gatekeeper', 'RegTech', 'Ex Ante Auto', 'Ex Post DeFi'],
    'Fixed Cost': ['Moderate', 'Moderate', 'High', 'Very High', 'Very High'],
    'Variable Cost': ['High', 'Moderate', 'Low', 'Very Low', 'High'],
    'Timing': ['Ex Post', 'Mixed', 'Real-time', 'Preventive', 'Ex Post'],
    'Authority': ['Human', 'Human', 'Algo+Human', 'Algorithmic', 'Human'],
}
fig, ax = plt.subplots(figsize=(12, 5))
ax.axis('off')
table = ax.table(cellText=[models_data['Fixed Cost'], models_data['Variable Cost'], models_data['Timing'], models_data['Authority']],
                 rowLabels=['Fixed Cost', 'Variable Cost', 'Timing', 'Authority'],
                 colLabels=models_data['Model'], loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.8)
ax.set_title('5 Regulatory Models Comparison', pad=20)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c13_regulatory_models.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C13: Regulatory Models")

# C15: Sovereignty Spectrum
fig, ax = plt.subplots(figsize=(12, 3))
gradient = np.linspace(0, 1, 256).reshape(1, -1)
ax.imshow(gradient, aspect='auto', cmap='RdYlGn_r', extent=[0, 10, 0, 1])
ax.set_xlim(0, 10)
ax.set_ylim(-0.5, 1.5)
for x in [2, 5, 8]: ax.axvline(x, color='white', linewidth=2, linestyle='--')
labels = [(1, '"Code is Law"\nNo Intervention'), (3.5, 'Minimal\nGovernance'), (6.5, 'Optimistic\nFreeze'), (9, '"Law as Code"\nFull Compliance')]
for x, label in labels:
    ax.text(x, 1.2, label, ha='center', va='bottom', fontsize=10)
ax.set_yticks([])
ax.set_xticks([])
ax.set_title('Sovereignty Risk Spectrum')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c15_sovereignty_spectrum.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C15: Sovereignty Spectrum")

# C22: LIF Stack
fig, ax = plt.subplots(figsize=(10, 8))
layers = [('L3: Legal Layer', 'DUNA/LLC Entity', COLORS['purple'], 0.85),
          ('L2: Safety Layer', 'Optimistic Freeze + Insurance', COLORS['success'], 0.65),
          ('L1: Consensus Layer', 'Base Protocol', COLORS['dark_blue'], 0.45),
          ('L0: Infrastructure', 'Nodes, Validators, RPC', COLORS['mid_blue'], 0.25)]
for name, desc, color, y in layers:
    ax.add_patch(mpatches.FancyBboxPatch((0.1, y - 0.08), 0.8, 0.15, boxstyle="round,pad=0.02", facecolor=color, alpha=0.7))
    ax.text(0.5, y, f'{name}\n{desc}', ha='center', va='center', fontsize=11, fontweight='bold', color='white')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('Complete LIF Stack')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c22_lif_stack.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C22: LIF Stack")

# C25: ZKP Flow
fig, ax = plt.subplots(figsize=(12, 4))
steps = [('Prover', 'Generates proof', COLORS['dark_blue']), ('Proof', 'ZK-SNARK', COLORS['success']),
         ('Verifier', 'On-chain contract', COLORS['purple']), ('Result', 'Allow/Block', COLORS['danger'])]
for i, (name, desc, color) in enumerate(steps):
    x = i / (len(steps) - 1) * 0.8 + 0.1
    ax.add_patch(mpatches.Circle((x, 0.5), 0.08, facecolor=color, alpha=0.7))
    ax.text(x, 0.5, name, ha='center', va='center', fontsize=10, fontweight='bold', color='white')
    ax.text(x, 0.25, desc, ha='center', va='top', fontsize=9)
    if i < len(steps) - 1:
        ax.annotate('→', ((x + (i+1)/(len(steps)-1)*0.8+0.1)/2, 0.5), fontsize=20, ha='center')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('ZKP Compliance Architecture')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c25_zkp_flow.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C25: ZKP Flow")

# C26: Optimistic Freeze
fig, ax = plt.subplots(figsize=(12, 4))
steps = [('Trigger', 'Anomaly\nDetected', COLORS['warning']), ('Pause', 'Immediate\nFreeze', COLORS['danger']),
         ('Vote', 'DAO/Council\nVote', COLORS['purple']), ('Outcome', 'Confirm or\nSlash Bond', COLORS['success'])]
for i, (name, desc, color) in enumerate(steps):
    x = i / (len(steps) - 1) * 0.8 + 0.1
    ax.add_patch(mpatches.FancyBboxPatch((x - 0.08, 0.35), 0.16, 0.3, boxstyle="round,pad=0.02", facecolor=color, alpha=0.7))
    ax.text(x, 0.5, f'{name}\n{desc}', ha='center', va='center', fontsize=9, fontweight='bold', color='white')
    if i < len(steps) - 1:
        ax.annotate('→', ((x + (i+1)/(len(steps)-1)*0.8+0.1)/2, 0.5), fontsize=20, ha='center', color=COLORS['text'])
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('Optimistic Freeze Mechanism')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c26_optimistic_freeze.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C26: Optimistic Freeze")

print(f"\n  Part 2 complete: {chart_count} charts total")

  [8] C09: Freeze Timeline
  [9] C10: Architectures of Control


  [10] C13: Regulatory Models
  [11] C15: Sovereignty Spectrum


  [12] C22: LIF Stack
  [13] C25: ZKP Flow
  [14] C26: Optimistic Freeze

  Part 2 complete: 14 charts total


In [7]:
# ============================================================
# CHART GENERATION - Part 3: Data-Driven Charts
# ============================================================

# C02: Cumulative Losses
yearly_cum = df.groupby('year')['loss_usd'].sum().cumsum() / 1_000_000_000
fig, ax = plt.subplots(figsize=(10, 6))
ax.fill_between(yearly_cum.index, yearly_cum.values, color=COLORS['danger'], alpha=0.3)
ax.plot(yearly_cum.index, yearly_cum.values, color=COLORS['danger'], linewidth=3, marker='o')
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('$%.0fB'))
ax.set_title('Cumulative Crypto Exploit Losses')
ax.set_ylabel('Cumulative Loss (Billions USD)')
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c02_cumulative_losses.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C02: Cumulative Losses")

# C17: Top 10 Exploits
top10 = df.nlargest(10, 'loss_usd')[['protocol', 'loss_usd']].copy()
top10['loss_billions'] = top10['loss_usd'] / 1_000_000_000
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(top10['protocol'], top10['loss_billions'], color=COLORS['dark_blue'])
bars[0].set_color(COLORS['danger'])
ax.bar_label(bars, fmt=' $%.2fB', padding=3, color=COLORS['text'])
ax.invert_yaxis()
ax.set_title('Top 10 Largest Crypto Exploits (2014-2025)')
ax.set_xlabel('Loss (Billions USD)')
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c17_top10_exploits.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C17: Top 10 Exploits")

# C18: Vector Distribution
vectors = df['vector_category'].value_counts().head(10)
fig, ax = plt.subplots(figsize=(10, 8))
colors_pie = [COLORS['dark_blue'], COLORS['danger'], COLORS['mid_blue'], COLORS['warning'], 
              COLORS['success'], COLORS['light_blue'], COLORS['purple'], COLORS['pink']] * 2
ax.pie(vectors, labels=vectors.index, autopct='%1.1f%%', colors=colors_pie[:len(vectors)], startangle=90, textprops={'fontsize': 9})
ax.set_title('Attack Vector Distribution (Top 10)')
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c18_vector_distribution.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C18: Vector Distribution")

# C19: Chain Distribution
chains = df.groupby('chain')['loss_usd'].sum().sort_values(ascending=False).head(10) / 1_000_000_000
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(chains.index, chains.values, color=COLORS['mid_blue'])
ax.bar_label(bars, fmt='$%.1fB', padding=3, fontsize=8)
ax.set_title('Losses by Chain (Top 10)')
ax.set_ylabel('Loss (Billions USD)')
plt.xticks(rotation=45, ha='right')
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c19_chain_distribution.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C19: Chain Distribution")

# C20: Losses by Year
yearly_sum = df.groupby('year')['loss_usd'].sum() / 1_000_000_000
fig, ax = plt.subplots(figsize=(10, 6))
ax.fill_between(yearly_sum.index, yearly_sum.values, color=COLORS['light_blue'], alpha=0.5)
ax.plot(yearly_sum.index, yearly_sum.values, color=COLORS['dark_blue'], linewidth=2, marker='o')
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('$%.0fB'))
ax.set_title('Annual Crypto Exploit Losses')
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'c20_losses_by_year.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] C20: Losses by Year")

# A01: Scatter Plot
fig, ax = plt.subplots(figsize=(12, 8))
sizes = np.clip(df['loss_usd'] / 1_000_000 * 2, 10, 500)
ax.scatter(df['date'], df['loss_usd'], s=sizes, color=COLORS['dark_blue'], alpha=0.5, edgecolors='white', linewidth=0.5)
ax.set_yscale('log')
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: '${:,.0f}'.format(y)))
ax.set_title('Timeline of Individual Exploits (Log Scale)')
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'a01_exploit_scatter.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] A01: Exploit Scatter")

# A02: Pareto
sorted_df = df.sort_values(by='loss_usd', ascending=False).reset_index(drop=True)
sorted_df['cumulative_pct'] = 100 * sorted_df['loss_usd'].cumsum() / df['loss_usd'].sum()
sorted_df['protocol_pct'] = 100 * (sorted_df.index + 1) / len(sorted_df)
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(sorted_df['protocol_pct'], sorted_df['cumulative_pct'], color=COLORS['dark_blue'], linewidth=2)
ax.axhline(80, color=COLORS['danger'], linestyle='--', alpha=0.5)
x_80 = sorted_df[sorted_df['cumulative_pct'] >= 80]['protocol_pct'].iloc[0]
ax.axvline(x_80, color=COLORS['danger'], linestyle='--', alpha=0.5)
ax.fill_between(sorted_df[sorted_df['protocol_pct'] <= x_80]['protocol_pct'], 
                sorted_df[sorted_df['protocol_pct'] <= x_80]['cumulative_pct'], alpha=0.3, color=COLORS['danger'])
ax.text(x_80 + 2, 10, f'Top {x_80:.1f}% = 80% Loss', color=COLORS['text'], fontsize=11)
ax.set_title('Pareto Analysis: Concentration of Losses')
ax.set_xlabel('% of Protocols')
ax.set_ylabel('Cumulative % of Loss')
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'a02_pareto.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] A02: Pareto Analysis")

# A03: Heatmap
pivot = df.pivot_table(index='month', columns='year', values='loss_usd', aggfunc='count', fill_value=0)
fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(pivot, cmap='Blues', linewidths=.5, ax=ax, cbar_kws={'label': 'Exploit Count'})
ax.set_title('Exploit Frequency Heatmap')
ax.set_yticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'a03_heatmap.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] A03: Heatmap")

# Monthly Trends 2024-2025
recent = df[df['year'] >= 2024].copy()
if not recent.empty:
    recent['year_month'] = recent['date'].dt.to_period('M')
    monthly = recent.groupby('year_month')['loss_usd'].sum() / 1_000_000
    monthly.index = monthly.index.astype(str)
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.fill_between(monthly.index, monthly.values, color=COLORS['danger'], alpha=0.3)
    ax.plot(monthly.index, monthly.values, color=COLORS['danger'], marker='o')
    ax.set_title('Monthly Loss Trends (2024-2025)')
    ax.set_ylabel('Loss (Millions USD)')
    plt.xticks(rotation=45, ha='right')
    for s in ['top', 'right']: ax.spines[s].set_visible(False)
    plt.tight_layout()
    plt.savefig(os.path.join(VIZ_PATH, 'monthly_trends_2024_2025.png'))
    plt.close()
    chart_count += 1
    print(f"  [{chart_count}] Monthly Trends 2024-2025")

# LIF Addressable
fig, ax = plt.subplots(figsize=(8, 6))
lif_losses = df[df['is_lif_relevant']]['loss_usd'].sum() / 1_000_000_000
other_losses = df[~df['is_lif_relevant']]['loss_usd'].sum() / 1_000_000_000
bars = ax.bar(['LIF-Addressable\nExploits', 'Other Exploits'], [lif_losses, other_losses], 
              color=[COLORS['success'], COLORS['mid_blue']])
ax.bar_label(bars, fmt='$%.1fB', padding=3)
ax.set_ylabel('Total Loss (Billions USD)')
ax.set_title('LIF-Addressable vs Other Exploits')
for s in ['top', 'right']: ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(VIZ_PATH, 'lif_addressable.png'))
plt.close()
chart_count += 1
print(f"  [{chart_count}] LIF Addressable")

print(f"\n✓ Total charts generated: {chart_count}")

  [15] C02: Cumulative Losses


  [16] C17: Top 10 Exploits
  [17] C18: Vector Distribution


  [18] C19: Chain Distribution


  [19] C20: Losses by Year


  [20] A01: Exploit Scatter


  [21] A02: Pareto Analysis


  [22] A03: Heatmap
  [23] Monthly Trends 2024-2025


  [24] LIF Addressable

✓ Total charts generated: 24


In [8]:
# ============================================================
# EXPORT RICH JSON STATISTICS
# ============================================================
print("\n" + "="*60)
print("EXPORTING JSON STATISTICS")
print("="*60)

def sanitize_for_json(obj):
    if hasattr(obj, 'isoformat'): return obj.isoformat()
    if hasattr(obj, 'item'): return obj.item()  # numpy types
    return str(obj)

def sanitize_dict(d):
    return {str(k): sanitize_for_json(v) if not isinstance(v, dict) else sanitize_dict(v) for k, v in d.items()}

lif_relevant_exploits = int(df['is_lif_relevant'].sum())
lif_relevant_loss_usd = float(df[df['is_lif_relevant']]['loss_usd'].sum())

stats = {
    'summary': {
        'total_exploits': int(len(df)),
        'total_loss_usd': float(df['loss_usd'].sum()),
        'lif_relevant_exploits': lif_relevant_exploits,
        'lif_relevant_loss_usd': lif_relevant_loss_usd,
        'top_protocol': df.nlargest(1, 'loss_usd')['protocol'].iloc[0],
        'date_range': {'start': str(df['date'].min().date()), 'end': str(df['date'].max().date())},
        'generated_at': datetime.now().isoformat(),
    },
    'top_10_exploits': [],
    'yearly_losses': sanitize_dict(df.groupby('year')['loss_usd'].sum().to_dict()),
    'cumulative_losses': sanitize_dict(df.groupby('year')['loss_usd'].sum().cumsum().to_dict()),
    'vector_distribution': df['vector_category'].value_counts().head(15).to_dict(),
    'chain_distribution': sanitize_dict(df.groupby('chain')['loss_usd'].sum().sort_values(ascending=False).head(10).to_dict()),
    'external_data': {
        'charoenwong': CHAROENWONG_DATA,
        'bybit': BYBIT_DATA,
        'anthropic': ANTHROPIC_DATA,
        'recovery': RECOVERY_DATA,
    },
    'data_sources': ['charoenwong_bernardi_table.txt', 'rekt_database_raw.txt',
                     'rekt_news_extra.txt', 'defihacklabs_incidents.json'],
}

# Top 10 exploits
for _, row in df.nlargest(10, 'loss_usd').iterrows():
    stats['top_10_exploits'].append({
        'protocol': row['protocol'],
        'loss_usd': float(row['loss_usd']),
        'date': row['date'].isoformat() if pd.notna(row['date']) else None,
        'vector_category': row['vector_category'],
        'chain': row['chain'],
        'is_lif_relevant': bool(row['is_lif_relevant']),
    })

# Monthly trends
if not recent.empty:
    stats['monthly_trends_2024_2025'] = sanitize_dict(
        recent.groupby(recent['date'].dt.to_period('M'))['loss_usd'].sum().to_dict()
    )

with open(JSON_PATH, 'w') as f:
    json.dump(stats, f, indent=2, default=str)

print(f"✓ JSON stats saved: {JSON_PATH}")


EXPORTING JSON STATISTICS
✓ JSON stats saved: /Users/elemoghenekaro/Desktop/tasks/legitimate-intervention-framework/data/refined/lif_stats.json


In [ ]:
# ============================================================
# PART 4: ELEM NIMROD PAPER CALIBRATION METRICS
# ============================================================
print("\n" + "="*60)
print("CALIBRATION METRICS")
print("="*60)

METRICS_PATH = os.path.join(BASE_DIR, 'data/refined/lif_intervention_metrics.csv')
if os.path.exists(METRICS_PATH):
    metrics_df = pd.read_csv(METRICS_PATH)
    print(f"Loaded calibration metrics: {len(metrics_df)} incidents")

    # C27: Time to Containment vs Loss Prevented
    fig, ax = plt.subplots(figsize=(10, 6))
    # Filter for incidents with valid time and prevention data
    plot_df = metrics_df[metrics_df['time_to_contain_min'] > 0].copy()
    
    sc = ax.scatter(plot_df['time_to_contain_min'], 
                   plot_df['loss_prevented_usd'] / 1e6, 
                   s=plot_df['containment_success_pct']*3, 
                   c=COLORS['dark_blue'], alpha=0.7)
    
    ax.set_xscale('log')
    ax.set_xlabel('Time to Containment (Minutes)')
    ax.set_ylabel('Loss Prevented ($M)')
    ax.set_title('Intervention Efficacy: Speed vs Value Preserved')
    
    # Annotate points
    for _, row in plot_df.iterrows():
        ax.annotate(row['protocol'], 
                   (row['time_to_contain_min'], row['loss_prevented_usd']/1e6), 
                   fontsize=8, xytext=(5, 5), textcoords='offset points')
        
    plt.tight_layout()
    plt.savefig(os.path.join(VIZ_PATH, 'c27_intervention_efficacy.png'))
    plt.close()
    print("  [25] C27: Intervention Efficacy")
    chart_count += 1
else:
    print(f"Warning: {METRICS_PATH} not found")


In [10]:
# ============================================================
# SUMMARY
# ============================================================
print("\n" + "="*60)
print("COMPLETE!")
print("="*60)
print(f"  Records cleaned: {len(df)}")
print(f"  Total loss: ${df['loss_usd'].sum():,.0f}")
print(f"  Charts generated: {chart_count}")
print(f"  LIF-relevant: {df['is_lif_relevant'].sum()} exploits (${df[df['is_lif_relevant']]['loss_usd'].sum():,.0f})")
print(f"\nFiles created:")
print(f"  - {CLEANED_CSV}")
print(f"  - {JSON_PATH}")
print(f"  - {chart_count} PNG files in {VIZ_PATH}")
print("="*60)


COMPLETE!
  Records cleaned: 763
  Total loss: $91,317,759,722
  Charts generated: 24
  LIF-relevant: 438 exploits ($10,759,673,502)

Files created:
  - /Users/elemoghenekaro/Desktop/tasks/legitimate-intervention-framework/data/refined/lif_exploits_cleaned.csv
  - /Users/elemoghenekaro/Desktop/tasks/legitimate-intervention-framework/data/refined/lif_stats.json
  - 24 PNG files in /Users/elemoghenekaro/Desktop/tasks/legitimate-intervention-framework/visualizations
